# FIFA World Cup 2022 - selenium scraper

In [ ]:
!pip install selenium==4.4.3

In [ ]:
!pip install pymongo

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
print(selenium.__version__)

4.4.3


In [3]:
import pandas as pd
import numpy as np
import json
import os
import tqdm
import shutil
import zipfile
from zipfile import ZipFile
import requests

import datetime
from datetime import datetime
from datetime import date
from tqdm import tqdm
import time
import warnings

import pymongo
from pymongo import MongoClient
warnings.filterwarnings('ignore')

In [4]:
FIFA_homepage = 'https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/'
FIFA_scores0 = 'scores-fixtures?country=IT&wtw-filter=ALL'
FIFA_website = FIFA_homepage + FIFA_scores0
print(FIFA_website)

https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/scores-fixtures?country=IT&wtw-filter=ALL


In [5]:
first_path = r'XXXXXXXXXXXXXXXXXXXX'

path_desktop0 = 'Desktop'
path_download0 = 'Download'

path_desktop = os.path.join(first_path, path_desktop0)
path_download = os.path.join(first_path, path_download0)

driver0 = os.path.join(path_desktop
                       ,'msedgedriver108'
                       ,'msedgedriver'
                      )
driver0

'XXXXXXXXXXXXXXXXXXXX\\Desktop\\msedgedriver108\\msedgedriver'

In [7]:
driver = webdriver.Edge(executable_path=driver0)

In [8]:
FIFA_teams_website = "https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/teams"
driver.get(FIFA_teams_website)

In [9]:
time.sleep(3)

## Reject all cookies

In [10]:
# cookies pop-up --> reject all cookies

delay = 10 #seconds

try:
    cookies = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((By.ID, 
                'onetrust-reject-all-handler'))).click()
except TimeoutException:
    print("no cookies pop-up")

no cookies pop-up


In [11]:
time.sleep(3)

## World Cup 2022 - Teams

In [12]:
#FIFA_teams_website = "https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/teams"
#driver.get(FIFA_teams_website)

teams0_short_names = WebDriverWait(driver, delay).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 
                "flag-with-info_flagAbbr__Dn4Ab")))

teams0_short_names1 = []
for i in range(len(teams0_short_names)):
    teams0_short_names1.append(teams0_short_names[i].text)
    


teams0_long_names = WebDriverWait(driver, delay).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 
                "flag-with-info_flagCountry__Yw8QR")))

teams0_long_names1 = []
for i in range(len(teams0_long_names)):
    teams0_long_names1.append(teams0_long_names[i].text)
    
    
df_team_names = pd.DataFrame(teams0_short_names1,
                            columns=['team_short_name'])
df_team_names['team_short_name'] = df_team_names['team_short_name'].str[1:-1]
df_team_names['team_long_name'] = teams0_long_names1
df_team_names['team_long_name_uppercase'] = df_team_names['team_long_name'].str.upper()

df_team_names

,team_short_name,team_long_name,team_long_name_uppercase
0,ARG,Argentina,ARGENTINA
1,AUS,Australia,AUSTRALIA
2,BEL,Belgium,BELGIUM
3,BRA,Brazil,BRAZIL
4,CMR,Cameroon,CAMEROON
5,CAN,Canada,CANADA
6,CRC,Costa Rica,COSTA RICA
7,CRO,Croatia,CROATIA
8,DEN,Denmark,DENMARK
9,ECU,Ecuador,ECUADOR


## Let's collect the URL of every single game listed on the webpage

In [13]:
driver.get(FIFA_website)

In [14]:
time.sleep(5)

In [15]:
# cookies pop-up --> reject all cookies

delay = 10 #seconds

try:
    cookies = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((By.ID, 
                'onetrust-reject-all-handler'))).click()
except TimeoutException:
    print("no cookies pop-up")

In [16]:
# all matches have the following URL in common
# we will search for all the elements containing this URL and get a list of all the football games

match_url0 = 'fifaplus/en/match-centre/match/17/255711/' #285063/'

In [17]:
match_url1 = "//a[contains(@href, '{0}')]".format(match_url0)
match_url1

"//a[contains(@href, 'fifaplus/en/match-centre/match/17/255711/')]"

In [18]:
games_world_cup_2022_selenium_elements_list = WebDriverWait(driver, delay).until(
                EC.presence_of_all_elements_located((By.XPATH, match_url1)))

print(type(games_world_cup_2022_selenium_elements_list))
print('')
print('number of games played during the tournament: ',
      len(games_world_cup_2022_selenium_elements_list)) #number of games played in the tournament

<class 'list'>

number of games played during the tournament:  64


In [19]:
# the list 'games_world_cup_2022_URL_list' contains the list of URL of every football game

games_world_cup_2022_URL_list = []

for i in tqdm(range(len(games_world_cup_2022_selenium_elements_list))):
    games_world_cup_2022_URL = games_world_cup_2022_selenium_elements_list[
                                        i].get_attribute('href')
    games_world_cup_2022_URL_list.append(games_world_cup_2022_URL)

100%|██████████| 64/64 [00:00<00:00, 72.56it/s]


In [20]:
len(games_world_cup_2022_URL_list) == len(games_world_cup_2022_selenium_elements_list)

True

In [21]:
games_world_cup_2022_URL_list[0:2]

['https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285063/400128082',
 'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285063/400235458']

In [22]:
games_world_cup_2022_URL_list[-5:]

['https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285074/400128138',
 'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285075/400128143',
 'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285075/400128142',
 'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285076/400128144',
 'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285077/400128145']

In [23]:
games_world_cup_2022_URL_df = pd.DataFrame(games_world_cup_2022_URL_list,
                                          columns=['URL'])
games_world_cup_2022_URL_df['ID_type_of_game'] = games_world_cup_2022_URL_df['URL'].apply(lambda x:x[62:68])
games_world_cup_2022_URL_df['ID_game'] = games_world_cup_2022_URL_df['URL'].apply(lambda x:x[69:78])
games_world_cup_2022_URL_df.head(3)

,URL,ID_type_of_game,ID_game
0,https://www.fifa.com/fifaplus/en/match-centre/...,285063,400128082
1,https://www.fifa.com/fifaplus/en/match-centre/...,285063,400235458
2,https://www.fifa.com/fifaplus/en/match-centre/...,285063,400235449


In [24]:
games_world_cup_2022_URL_df.iloc[0,0]

'https://www.fifa.com/fifaplus/en/match-centre/match/17/255711/285063/400128082'

## Now that we have collected all the URL, we will open each webpage and scrape the available data

In [25]:
possesso_palla_casa =           "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div[2]/div/div[2]/div[1]"
possesso_palla_trasferta =      "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div[2]/div/div[2]/div[3]"
gol_totale_casa =               "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div[4]/div[1]/div[2]/div[1]"
gol_totale_trasferta =          "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div[4]/div[1]/div[2]/div[3]"
tiri_totale_casa =              "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[3]/div[2]/div[1]/div[2]/div[1]"
tiri_totale_trasferta =         "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[3]/div[2]/div[1]/div[2]/div[3]"
tiri_in_porta_casa =            "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div[2]/div[1]"
tiri_in_porta_trasferta =       "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div[2]/div[3]" 
ammonizioni_casa =              "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[1]/div[2]/div[1]" 
ammonizioni_trasferta =         "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[1]/div[2]/div[3]" 
espulsioni_casa =               "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[2]/div[2]/div[1]" 
espulsioni_trasferta =          "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[2]/div[2]/div[3]" 
falli_subiti_casa =             "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[3]/div[2]/div[1]" 
falli_subiti_trasferta =        "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[3]/div[2]/div[3]" 
fuorigioco_casa =               "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[4]/div[2]/div[1]" 
fuorigioco_trasferta =          "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[8]/div[2]/div[4]/div[2]/div[3]"
passaggi_casa =                 "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[1]/div[2]/div[1]" 
passaggi_trasferta =            "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[1]/div[2]/div[3]"
passaggi_completati_casa =      "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[2]/div[2]/div[1]" 
passaggi_completati_trasferta = "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[2]/div[2]/div[3]"
cross_casa =                    "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[3]/div[2]/div[1]" 
cross_trasferta =               "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[3]/div[2]/div[3]" 
cross_completati_casa =         "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[4]/div[2]/div[1]" 
cross_completati_trasferta =    "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[9]/div[2]/div[4]/div[2]/div[3]" 
corner_casa =                   "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[1]/div[2]/div[1]" 
corner_trasferta =              "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[1]/div[2]/div[3]" 
calci_piazzati_casa =           "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[2]/div[2]/div[1]" 
calci_piazzati_trasferta =      "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[2]/div[2]/div[3]" 
rigori_segnati_casa =           "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[3]/div[2]/div[1]" 
rigori_segnati_trasferta =      "/html/body/div[1]/main/div/div[3]/div/div[2]/div/div/div[1]/div/div[10]/div[2]/div[3]/div[2]/div[3]"

list_statistiche = [possesso_palla_casa     
                    ,possesso_palla_trasferta
                    ,gol_totale_casa
                    ,gol_totale_trasferta
                    ,tiri_totale_casa
                    ,tiri_totale_trasferta
                    ,tiri_in_porta_casa
                    ,tiri_in_porta_trasferta 
                    ,ammonizioni_casa 
                    ,ammonizioni_trasferta 
                    ,espulsioni_casa
                    ,espulsioni_trasferta
                    ,falli_subiti_casa
                    ,falli_subiti_trasferta
                    ,fuorigioco_casa
                    ,fuorigioco_trasferta
                    ,passaggi_casa
                    ,passaggi_trasferta
                    ,passaggi_completati_casa
                    ,passaggi_completati_trasferta
                    ,cross_casa 
                    ,cross_trasferta
                    ,cross_completati_casa
                    ,cross_completati_trasferta
                    ,corner_casa
                    ,corner_trasferta
                    ,calci_piazzati_casa
                    ,calci_piazzati_trasferta
                    ,rigori_segnati_casa
                    ,rigori_segnati_trasferta
                    ]



list_statistiche_stringhe = ['percentuale_possesso_palla_casa'     
                        ,'percentuale_possesso_palla_trasferta'
                        ,'gol_totale_casa'
                        ,'gol_totale_trasferta'
                        ,'tiri_totale_casa'
                        ,'tiri_totale_trasferta'
                        ,'tiri_in_porta_casa'
                        ,'tiri_in_porta_trasferta' 
                        ,'ammonizioni_casa' 
                        ,'ammonizioni_trasferta' 
                        ,'espulsioni_casa'
                        ,'espulsioni_trasferta'
                        ,'falli_subiti_casa'
                        ,'falli_subiti_trasferta'
                        ,'fuorigioco_casa'
                        ,'fuorigioco_trasferta'
                        ,'passaggi_casa'
                        ,'passaggi_trasferta'
                        ,'passaggi_completati_casa'
                        ,'passaggi_completati_trasferta'
                        ,'cross_casa'
                        ,'cross_trasferta'
                        ,'cross_completati_casa'
                        ,'cross_completati_trasferta'
                        ,'corner_casa'
                        ,'corner_trasferta'
                        ,'calci_piazzati_casa'
                        ,'calci_piazzati_trasferta'
                        ,'rigori_segnati_casa'
                        ,'rigori_segnati_trasferta'
                        ]

list_names00 = ['tipo_di_partita',
            'squadra_in_casa',
            'squadra_in_trasferta',
            'datetime_kick_off',
            'gol_segnati_casa',
            'gol_segnati_trasferta',
            'rigori_finali_segnati_casa',
            'rigori_finali_segnati_trasferta'
            ]


In [26]:
def match_header_function(driver):
    #match_header0 = WebDriverWait(driver, delay).until(
    #        EC.presence_of_element_located((By.CLASS_NAME, 
    #                                        "ff-p-0"))).text
    match_header0 = driver.find_element(By.CLASS_NAME, 
                                            "ff-p-0").text
    match_header0 = match_header0.split('\n')
    type_of_game = match_header0[0]
    teams_ID_list_short = list(df_team_names['team_short_name'].values)
    teams_ID_list_long = list(df_team_names['team_long_name_uppercase'].values)

    teams00 = []
    for i in range(len(match_header0[0:7])):
        if (match_header0[i] in teams_ID_list_short) or (match_header0[i] in teams_ID_list_long):
            teams00.append(match_header0[i])

    home_team = teams00[0]
    away_team = teams00[1]

    kick_off_datetime = []
    dot = '  •  '
    for i in range(len(match_header0[0:3])):
        if dot in match_header0[i]:
            kick_off_datetime.append(match_header0[i])


    #kick_off_datetime = kick_off_datetime[0].split(dot)
    #print(kick_off_datetime)

    kick_off_datetime = kick_off_datetime[0].split(' ')
    hour0 = kick_off_datetime[0].zfill(2)
    minutes0 = kick_off_datetime[2].zfill(2)
    day0 = kick_off_datetime[6].zfill(2)
    month0 = kick_off_datetime[7]
    year0 = kick_off_datetime[8].zfill(4)

    if month0 == 'DEC':
        month0 = 12
    elif month0 == 'NOV':
        month0 = 11
    #elif month0 == 'OCT':
    #    month0 = 10


    kick_off_datetime = '{0}-{1}-{2} {3}:{4}'.format(year0,
                                                   month0,
                                                   day0,
                                                   hour0,
                                                   minutes0
                                                  )

    kick_off_datetime = datetime.strptime(kick_off_datetime, '%Y-%m-%d %H:%M')


    #print('home team: ', home_team)
    #print('away team: ', away_team)

    #print('type of game: ',type_of_game)
    #print('kick-off datetime: ',kick_off_datetime)

    hyphen = r' - ' 
    if hyphen in match_header0[4]:
        score = match_header0[4].split(hyphen)
        home_teams_goals = int(score[0])
        away_teams_goals = int(score[1][2:])
        penalties_scored_home_team = int(match_header0[3][1:-1])
        penalties_scored_away_team = int(match_header0[5][1:-1])
    elif hyphen in match_header0[3]:
        score = match_header0[3].split(hyphen)
        home_teams_goals = int(score[0])
        away_teams_goals = int(score[1][2:])
        penalties_scored_home_team = 0
        penalties_scored_away_team = 0
    #print('home team goals: ',home_teams_goals)
    #print('away team goals: ',away_teams_goals)
    #print('penalties scored home team: ',penalties_scored_home_team)
    #print('penalties scored away team: ',penalties_scored_away_team)
    #print('')
    
       
    list_values00 = [type_of_game,
                     home_team,
                     away_team,
                     kick_off_datetime,
                     home_teams_goals,
                     away_teams_goals,
                     penalties_scored_home_team,
                     penalties_scored_away_team
                        ]

    df_header00 = pd.DataFrame(list_values00,
                               columns=['valori'],
                              index=list_names00)
    
    df_header00 = df_header00.T.reset_index(drop=True)
    
    df_header00['nome_partita'] = df_header00[
                'squadra_in_casa'] + "-" + df_header00[
                'squadra_in_trasferta']

    df_header00['kick_off_day'] = df_header00['datetime_kick_off'].dt.date
    df_header00['kick_off_day'] = pd.to_datetime(df_header00['kick_off_day'])
    df_header00['datetime_kick_off_string'] = df_header00['datetime_kick_off'].astype(str)
    df_header00['kick_off_day_string'] = df_header00['kick_off_day'].astype(str)
    
    colonne_numeriche00 = ['gol_segnati_casa'
                           ,'gol_segnati_trasferta'
                           ,'rigori_finali_segnati_casa'
                           ,'rigori_finali_segnati_trasferta'
                        ]

    for i in colonne_numeriche00:
        df_header00[i] = df_header00[i].astype(int)
        
    return df_header00

In [27]:
#list(games_world_cup_2022_URL_df['URL'].values)
#games_world_cup_2022_URL_list
print(len(games_world_cup_2022_URL_list[:]))

64


In [28]:
def gol_scored_function(driver,
                       button_XPATH,
                       gol_squadra_in_casa_XPATH,
                       gol_squadra_in_trasferta_XPATH):
    button = driver.find_element(By.XPATH, button_XPATH)
    button.click()
    time.sleep(2)


    xpath_list_gol0 = [gol_squadra_in_casa_XPATH,
                      gol_squadra_in_trasferta_XPATH]

    xpath_list_gol0_words = ['casa',
                            'trasferta']
    dataframe_list_goals = []

    for k, j in zip(xpath_list_gol0, xpath_list_gol0_words): 
        gol_squadra_element = driver.find_element(By.XPATH, k)
        gol_information_list = gol_squadra_element.text.split('\n')

        gol_information_list01 = []
        for i in gol_information_list:
            if len(gol_information_list)>1:
                if i[0].isalpha():
                    #nella partita Argentina-Croazia, un allenatore croato è stato espulso
                    if not i.endswith("(Staff)"):
                        gol_information_list01.append(i)
        
        #if len(gol_information_list01)>0: 
        goal_df0 = pd.DataFrame(gol_information_list01,
                               columns=['goal_scorer']).reset_index()
        goal_df0['length'] = len(goal_df0) - goal_df0['index']
        goal_df0['count'] = 'gol_{}_'.format(j)
        goal_df0['count1'] = goal_df0['count'] + goal_df0['length'].astype(str)
        goal_df0 = goal_df0[['count1','goal_scorer']]
        dataframe_list_goals.append(goal_df0)

    dataframe_goals = pd.concat(dataframe_list_goals,
                               ignore_index=True)
    dataframe_goals = dataframe_goals.set_index('count1').T.reset_index(drop=True)
    return dataframe_goals

In [29]:
def statische_partita_function(driver,
                             STATS_button_XPATH):
    STATS_button = driver.find_element(By.XPATH, STATS_button_XPATH)
    STATS_button.click() 
    time.sleep(3)
    statistiche_partita_lista00 = []
    list_valori_in_percentuale = [possesso_palla_casa     
                                  ,possesso_palla_trasferta
                                 ]
    for i in list_statistiche:
        element_text = driver.find_element("xpath",i).text
        if element_text[-1] == r"%":
            element_text = (element_text)[:-1]
        statistiche_partita_lista00.append(int(element_text))


    statistiche_df = pd.DataFrame(statistiche_partita_lista00,
                                  columns=['valori_statistiche'],
                                  index=list_statistiche_stringhe
                                 )
    test = statistiche_df.T.reset_index(drop=True)
    #time.sleep(3)
    return test

In [30]:
def cartellini_rossi_casa_function(final_df):
    if final_df['espulsioni_casa'].values[0] == 1:
        nome_ultimo_gol_casa = []
        for i in list(final_df.columns):
            if i.startswith('gol_casa'):
                nome_ultimo_gol_casa.append(i)
        final_df = final_df.rename(
            columns={nome_ultimo_gol_casa[0]:'cartellino_rosso_casa_1'})
    return final_df

In [31]:
list_of_games_stats_dataframes = []

In [32]:
print(len(list_of_games_stats_dataframes))

0


In [33]:
print(np.arange(0,10,1))  #  0:10 #
print(np.arange(10,20,1)) # 10:20 #
print(np.arange(20,30,1)) # 20:30 #
print(np.arange(30,40,1)) # 30:40 #
print(np.arange(40,50,1)) # 40:50 #
print(np.arange(50,60,1)) # 50:60 #
print(np.arange(60,64,1)) # 60:64 #

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63]


In [37]:
try:
    driver.quit()
except:
    pass

driver = webdriver.Edge(executable_path=driver0)
#time.sleep(3)

STATS_button_XPATH = "/html/body/div[1]/main/div/div[3]/div/div[1]/div/div[4]"
button_XPATH = '/html/body/div[1]/main/div/div[1]/div/div/button'
gol_squadra_in_casa_XPATH =      "/html/body/div[1]/main/div/div[1]/div/section/div[2]/div[1]/div[1]"
gol_squadra_in_trasferta_XPATH = "/html/body/div[1]/main/div/div[1]/div/section/div[2]/div[1]/div[3]"


for i in tqdm(games_world_cup_2022_URL_list[:]):
    driver.get(i)
    time.sleep(3)
    try:
        cookies0 = driver.find_element(By.ID, 'onetrust-reject-all-handler')
        #cookies0 = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div[1]/div/div[2]/div/button[2]")
        cookies0.click()
    except:
        pass
    time.sleep(1)
    df_header00 = match_header_function(driver)
    
    goal_df0 = gol_scored_function(driver,
                       button_XPATH,
                       gol_squadra_in_casa_XPATH,
                       gol_squadra_in_trasferta_XPATH)
    
    test = statische_partita_function(driver,
                             STATS_button_XPATH)
   
    final_df = pd.merge(df_header00,
                    test,
                    left_index=True,
                    right_index=True
                   )
    #time.sleep(5)
    final_df = pd.merge(final_df,
                        goal_df0,
                        left_index=True,
                        right_index=True
                       )
    
    final_df = cartellini_rossi_casa_function(final_df)
    
    initial_columns = list(final_df.columns)[0:4]
    print(final_df[initial_columns])
    print('')
    print('')
    
    list_of_games_stats_dataframes.append(final_df)
    time.sleep(1)
    

driver.quit()
print('') 
print('')
print('numero di partite salvate nella lista: ', 
      len(list_of_games_stats_dataframes))

  0%|          | 0/64 [00:00<?, ?it/s]

  tipo_di_partita squadra_in_casa squadra_in_trasferta   datetime_kick_off
0         Group A           QATAR              ECUADOR 2022-11-20 17:00:00




  2%|▏         | 1/64 [00:21<22:43, 21.64s/it]

  tipo_di_partita squadra_in_casa squadra_in_trasferta   datetime_kick_off
0         Group B         ENGLAND                 IRAN 2022-11-21 14:00:00




  3%|▎         | 2/64 [00:33<16:21, 15.84s/it]

...

            tipo_di_partita squadra_in_casa squadra_in_trasferta  \
0  Play-off for third place         CROATIA              MOROCCO   

    datetime_kick_off  
0 2022-12-17 16:00:00  




 98%|█████████▊| 63/64 [12:25<00:11, 11.57s/it]

  tipo_di_partita squadra_in_casa squadra_in_trasferta   datetime_kick_off
0           Final       ARGENTINA               FRANCE 2022-12-18 16:00:00




100%|██████████| 64/64 [12:36<00:00, 11.83s/it]




numero di partite salvate nella lista:  64


In [38]:
len(list_of_games_stats_dataframes)

64

In [39]:
games_stats_dataframes_concat = pd.concat(list_of_games_stats_dataframes, 
                                          ignore_index=True)

lista_completa_nomi_gol00 = []
lista_completa_nomi_gol00_index = []
list888 = list(games_stats_dataframes_concat.columns)
for i in range(len(list888)):
    if (list888[i].startswith('gol_casa')
       ) or (list888[i].startswith('gol_trasferta')) or (
        list888[i].startswith('cartellino_rosso_casa')):
            lista_completa_nomi_gol00.append(list888[i])
            lista_completa_nomi_gol00_index.append(i)

list888_01 = list888[0:(lista_completa_nomi_gol00_index[0])]
lista_completa_nomi_gol00.sort()
lista_aggiornata_nomi_colonne = list888_01 + lista_completa_nomi_gol00

games_stats_dataframes_concat = games_stats_dataframes_concat[
    lista_aggiornata_nomi_colonne]

games_stats_dataframes_concat.tail()

,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,kick_off_day,...,gol_casa_2,gol_casa_3,gol_casa_4,gol_casa_5,gol_casa_6,gol_casa_7,gol_trasferta_1,gol_trasferta_2,gol_trasferta_3,gol_trasferta_4
59,Quarter-final,ENGLAND,FRANCE,2022-12-10 20:00:00,1,2,0,0,ENGLAND-FRANCE,2022-12-10,...,NaN,NaN,NaN,NaN,NaN,NaN,Aurelien TCHOUAMENI,Olivier GIROUD,NaN,NaN
60,Semi-final,ARGENTINA,CROATIA,2022-12-13 20:00:00,3,0,0,0,ARGENTINA-CROATIA,2022-12-13,...,Julian ALVAREZ,Julian ALVAREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Semi-final,FRANCE,MOROCCO,2022-12-14 20:00:00,2,0,0,0,FRANCE-MOROCCO,2022-12-14,...,Randal KOLO MUANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Play-off for third place,CROATIA,MOROCCO,2022-12-17 16:00:00,2,1,0,0,CROATIA-MOROCCO,2022-12-17,...,Mislav ORSIC,NaN,NaN,NaN,NaN,NaN,Achraf DARI,NaN,NaN,NaN
63,Final,ARGENTINA,FRANCE,2022-12-18 16:00:00,3,3,4,2,ARGENTINA-FRANCE,2022-12-18,...,Angel DI MARIA,Lionel MESSI,NaN,NaN,NaN,NaN,Kylian MBAPPE,Kylian MBAPPE,Kylian MBAPPE,NaN


In [40]:
# partite senza cartellini rossi

games_stats_dataframes_concat1_senza_cartellini_rossi = games_stats_dataframes_concat[
    (games_stats_dataframes_concat['espulsioni_casa']==0) &
    (games_stats_dataframes_concat['espulsioni_trasferta']==0)]

In [41]:
# partite con cartellini rossi --> nella pagina web del sito FIFA,
# i nomi dei giocatori che hanno segnato un gol sono indicati
#nello stesso box dei giocatori che hanno preso un cartellino rosso

cartellini_rossi = games_stats_dataframes_concat[
    (games_stats_dataframes_concat['espulsioni_casa']>0) |
    (games_stats_dataframes_concat['espulsioni_trasferta']>0)].copy()

'''
cartellini_rossi = cartellini_rossi.rename(
    columns={"gol_0": "red_card_0"})
gol_names_list = []
gol_names_list00 = []
for i in list(cartellini_rossi.columns):
    if (i.startswith('gol_')) and (len(i)<=5):
        gol_names_list.append(i)
        qq = 'gol_' + str(int(i[-1])-1)
        gol_names_list00.append(qq)
dict_gol_names = dict(zip(gol_names_list, gol_names_list00))
cartellini_rossi = cartellini_rossi.rename(columns=dict_gol_names)
'''

cartellini_rossi[['tipo_di_partita',
                  'squadra_in_casa',
                  'squadra_in_trasferta',	
                  'datetime_kick_off',
                  'espulsioni_casa',
                  'espulsioni_trasferta',
                  'cartellino_rosso_casa_1',
                  'gol_casa_1',
                  'gol_casa_2',
                  'gol_casa_3',
                  ]]

,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,espulsioni_casa,espulsioni_trasferta,cartellino_rosso_casa_1,gol_casa_1,gol_casa_2,gol_casa_3
16,Group B,WALES,IRAN,2022-11-25 11:00:00,1,0,Wayne HENNESSEY,NaN,NaN,NaN
47,Group G,CAMEROON,BRAZIL,2022-12-02 20:00:00,1,0,Vincent ABOUBAKAR,Vincent ABOUBAKAR,NaN,NaN
57,Quarter-final,NETHERLANDS,ARGENTINA,2022-12-09 20:00:00,1,0,Denzel DUMFRIES,Wout WEGHORST,Wout WEGHORST,NaN
58,Quarter-final,MOROCCO,PORTUGAL,2022-12-10 16:00:00,1,0,Walid CHEDDIRA,Youssef EN NESYRI,NaN,NaN


In [42]:
games_stats_dataframes_concat.shape

(64, 54)

In [43]:
name_of_csv_file = '2022_qatar_world_cup_official_stats_FIFA.csv'
path_of_csv_file = os.path.join(path_download, name_of_csv_file)
games_stats_dataframes_concat.to_csv(path_or_buf=path_of_csv_file
                                    ,sep=';'
                                    ,header=True
                                    ,index=False)

# Inserimento su mongo

In [44]:
stats = games_stats_dataframes_concat.copy()

In [125]:
games_stats_dataframes_concat=games_stats_dataframes_concat.replace(to_replace="KOREA REPUBLIC-GHANA",
         value="KOREA-GHANA")#per l'integrazione con il ranking
games_stats_dataframes_concat=games_stats_dataframes_concat.replace(to_replace="KOREA REPUBLIC-PORTUGAL",
         value="KOREA-PORTUGAL")
games_stats_dataframes_concat=games_stats_dataframes_concat.replace(to_replace="BRAZIL-KOREA REPUBLIC",
         value="BRAZIL-KOREA")
games_stats_dataframes_concat=games_stats_dataframes_concat.replace(to_replace="URUGUAY-KOREA REPUBLIC",
         value="URUGUAY-KOREA")
#stats=stats.replace(to_replace="UNITED STATES-WALES",
 #        value="USA-WALES")#per l'integrazione con il ranking
#stats=stats.replace(to_replace="IRAN-UNITED STATES",
 #        value="IRAN-USA")
#stats=stats.replace(to_replace="NETHERLANDS-UNITED STATES",
#         value="NETHERLANDS-USA")
#stats=stats.replace(to_replace="ENGLAND-UNITED STATES",
 #        value="ENGLAND-USA")

In [126]:
games_stats_dataframes_concat

,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,kick_off_day,...,gol_casa_2,gol_casa_3,gol_casa_4,gol_casa_5,gol_casa_6,gol_casa_7,gol_trasferta_1,gol_trasferta_2,gol_trasferta_3,gol_trasferta_4
0,Group A,QATAR,ECUADOR,2022-11-20 17:00:00,0,2,0,0,QATAR-ECUADOR,2022-11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,Enner VALENCIA,Enner VALENCIA,NaN,NaN
1,Group B,ENGLAND,IRAN,2022-11-21 14:00:00,6,2,0,0,ENGLAND-IRAN,2022-11-21,...,Bukayo SAKA,Raheem STERLING,Bukayo SAKA,Marcus RASHFORD,Jack GREALISH,NaN,Mehdi TAREMI,Mehdi TAREMI,NaN,NaN
2,Group A,SENEGAL,NETHERLANDS,2022-11-21 17:00:00,0,2,0,0,SENEGAL-NETHERLANDS,2022-11-21,...,NaN,NaN,NaN,NaN,NaN,NaN,Cody GAKPO,Davy KLAASSEN,NaN,NaN
3,Group B,UNITED STATES,WALES,2022-11-21 20:00:00,1,1,0,0,UNITED STATES-WALES,2022-11-21,...,NaN,NaN,NaN,NaN,NaN,NaN,Gareth BALE,NaN,NaN,NaN
4,Group C,ARGENTINA,SAUDI ARABIA,2022-11-22 11:00:00,1,2,0,0,ARGENTINA-SAUDI ARABIA,2022-11-22,...,NaN,NaN,NaN,NaN,NaN,NaN,SALEH ALSHEHRI,SALEM ALDAWSARI,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Quarter-final,ENGLAND,FRANCE,2022-12-10 20:00:00,1,2,0,0,ENGLAND-FRANCE,2022-12-10,...,NaN,NaN,NaN,NaN,NaN,NaN,Aurelien TCHOUAMENI,Olivier GIROUD,NaN,NaN
60,Semi-final,ARGENTINA,CROATIA,2022-12-13 20:00:00,3,0,0,0,ARGENTINA-CROATIA,2022-12-13,...,Julian ALVAREZ,Julian ALVAREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Semi-final,FRANCE,MOROCCO,2022-12-14 20:00:00,2,0,0,0,FRANCE-MOROCCO,2022-12-14,...,Randal KOLO MUANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Play-off for third place,CROATIA,MOROCCO,2022-12-17 16:00:00,2,1,0,0,CROATIA-MOROCCO,2022-12-17,...,Mislav ORSIC,NaN,NaN,NaN,NaN,NaN,Achraf DARI,NaN,NaN,NaN


In [123]:
stats_json=games_stats_dataframes_concat.to_json(orient="records")
stats_json = json.loads(stats_json)
json.dumps(stats_json, indent=4)
stats_json[1]



{'tipo_di_partita': 'Group B',
 'squadra_in_casa': 'ENGLAND',
 'squadra_in_trasferta': 'IRAN',
 'datetime_kick_off': '2022-11-21 14:00:00',
 'gol_segnati_casa': 6,
 'gol_segnati_trasferta': 2,
 'rigori_finali_segnati_casa': 0,
 'rigori_finali_segnati_trasferta': 0,
 'nome_partita': 'ENGLAND-IRAN',
 'kick_off_day': '2022-11-21',
 'datetime_kick_off_string': '2022-11-21 14:00:00',
 'kick_off_day_string': '2022-11-21',
 'percentuale_possesso_palla_casa': 72,
 'percentuale_possesso_palla_trasferta': 19,
 'gol_totale_casa': 6,
 'gol_totale_trasferta': 2,
 'tiri_totale_casa': 13,
 'tiri_totale_trasferta': 8,
 'tiri_in_porta_casa': 7,
 'tiri_in_porta_trasferta': 3,
 'ammonizioni_casa': 0,
 'ammonizioni_trasferta': 2,
 'espulsioni_casa': 0,
 'espulsioni_trasferta': 0,
 'falli_subiti_casa': 9,
 'falli_subiti_trasferta': 14,
 'fuorigioco_casa': 2,
 'fuorigioco_trasferta': 2,
 'passaggi_casa': 809,
 'passaggi_trasferta': 224,
 'passaggi_completati_casa': 730,
 'passaggi_completati_trasferta': 154

In [127]:
test = []
for i in range(len(stats)):
    el = pd.DataFrame(stats.iloc[i , :]).T
    el = el.dropna(axis = 1)
    test.append(el)
type(test[0])


pandas.core.frame.DataFrame

In [128]:
test[6]

,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,kick_off_day,...,cross_casa,cross_trasferta,cross_completati_casa,cross_completati_trasferta,corner_casa,corner_trasferta,calci_piazzati_casa,calci_piazzati_trasferta,rigori_segnati_casa,rigori_segnati_trasferta
6,Group C,MEXICO,POLAND,2022-11-22 17:00:00,0,0,0,0,MEXICO-POLAND,2022-11-22,...,33,10,8,2,6,5,16,16,0,0


In [140]:
stats_json=[]
for i in range(0,len(test)):
    result=test[i].to_json(orient="records")
    parsed = json.loads(result)
    json.dumps(parsed, indent=4)
    stats_json.append(parsed[0])

stats_json[0]


{'tipo_di_partita': 'Group A',
 'squadra_in_casa': 'QATAR',
 'squadra_in_trasferta': 'ECUADOR',
 'datetime_kick_off': '2022-11-20 17:00:00',
 'gol_segnati_casa': 0,
 'gol_segnati_trasferta': 2,
 'rigori_finali_segnati_casa': 0,
 'rigori_finali_segnati_trasferta': 0,
 'nome_partita': 'QATAR-ECUADOR',
 'kick_off_day': '2022-11-20',
 'datetime_kick_off_string': '2022-11-20 17:00:00',
 'kick_off_day_string': '2022-11-20',
 'percentuale_possesso_palla_casa': 42,
 'percentuale_possesso_palla_trasferta': 50,
 'gol_totale_casa': 0,
 'gol_totale_trasferta': 2,
 'tiri_totale_casa': 5,
 'tiri_totale_trasferta': 6,
 'tiri_in_porta_casa': 0,
 'tiri_in_porta_trasferta': 3,
 'ammonizioni_casa': 4,
 'ammonizioni_trasferta': 2,
 'espulsioni_casa': 0,
 'espulsioni_trasferta': 0,
 'falli_subiti_casa': 15,
 'falli_subiti_trasferta': 15,
 'fuorigioco_casa': 3,
 'fuorigioco_trasferta': 4,
 'passaggi_casa': 450,
 'passaggi_trasferta': 480,
 'passaggi_completati_casa': 381,
 'passaggi_completati_trasferta': 4

In [141]:
connessione = pymongo.MongoClient("mongodb://localhost:27017/")

WorldCupDb = connessione["WorldCup"] #crea DB
WorldCupCollection = WorldCupDb["WorldCupStats"] #crea collezione all'interno del db
print(connessione.list_database_names())#si utilizza il db WorldCup


['WorldCup', 'admin', 'config', 'local']


In [142]:
for i in range(0,len(stats_json)):
    inserimento = WorldCupCollection.insert_one(stats_json[i])